In [11]:
import torch
import torch.nn.functional as F

In [7]:
words = open('names.txt', 'r').read().splitlines()
chars = sorted(set(''.join(words)))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [68]:
xs, ys = [], []
for w in words[:]:
    chs = ['.', '.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix12 = stoi[ch1] * 27 + stoi[ch2]
        xs.append(ix12)
        ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()

In [69]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((729, 27), generator=g, requires_grad=True)

In [70]:
xenc = F.one_hot(xs, num_classes=729).float()
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()

In [71]:
W.grad = None
loss.backward()

In [72]:
W.data += -0.1 * W.grad

In [73]:
print(loss.item())

3.8028225898742676


In [102]:
for k in range(1000):
    xenc = F.one_hot(xs, num_classes=729).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()

    W.grad = None
    loss.backward()

    W.data += -1 * W.grad

print(loss.item())

2.3648223876953125


In [33]:
print(loss.item())

2.853464365005493


In [104]:
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
    out = []
    i1, i2 = 0, 0
    while True:
        pair = i1 * 27 + i2
        logits = W[pair].unsqueeze(0)
        p = logits.softmax(dim=1).squeeze(0)
        i3 = torch.multinomial(p, num_samples=1).item()
        if i3 == 0:
            break
        else:
            out.append(itos[i3])
            i1, i2 = i2, i3
    print(''.join(out))

ammar
man
ayan
ali
susmore
